In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import ml models here
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier

from sklearn.decomposition import PCA
import plotly.express as px

---
# Gradient Descent for MLP with sigmoid activation and quartic loss

Question statement: \
Develop the training logic for a **SINGLE HIDDEN LAYER MLP** classifier using the sigmoid function as activation function and the sum of errors to the power of four (instead of the sum of squared errors) as the error function.

---
# Messy details of the netwok itself
Our goal is to define a neural network that defines a **function** 
 * taking $d$-dimensional input, i.e. each input $x = (x_1, \dots, x_d) = (x_i)_{i = 1, \dots, d}$ is a $d$-dimensional vector. 
 * and output $n$ dimensional vector
Meaning, out neural network here would be a function of the following form.
$$
f: \mathbb{R}^d \to \mathbb{R}^n\\
f(\mathbf{x}) = f(x_1, \dots, x_d) = \left(f_1(\mathbf{x}), \dots, f_n(\mathbf{x})\right)
$$

**Input layer to hidden layer** 

Since we have $d$-dimensional input, the input layer has $d$ nodes. Now, since we are only implementing single-layer feedforward network, we only have 1-hidden layer with a size of $M$. So, each node in the hidden layer is taking all $d$ values coming from the input layer and computing the function
$$ \text{HiddenNode}_j(x_1, \dots, x_d) = \sigma\left(\sum_{i = 1}^d W^{(1)}_{ji}x_i + b^{(1)}_{j}\right)$$
Note:
 * The question asked us to use sigmoid activation function, i.e. 
 $$\sigma(x) = \frac{1}{1 - e^{-x}}$$
 * Note that the activation function only cares about a **scalar** input!! It takes in one number and output one number. 
 * the superscript in $W^{(1)}, b^{(1)}$ is to remind ourselves that we are going from layer $0$ (the input) to layer $1$. 
 * I am fixing the indexing variable here so that you know what variable correspond to which layer. 
 * $i$ is the index for the input layer, $i = 1, \dots d$. 
 * $j$ is the index for the hidden layer, $j = 1, \dots M$. So the above expression is repeated with different weights $W_{ji}$ and biases $b_{j}$ for each $j = 1, \dots, M$. 
 * We use capital letter $W$ because it is going to look a whole lot like a marix later on. 
 * The reason the index $ji$ is labelled in that order is also because they correspond to the row and column index of the matrix later on. 
 

**Hidden layer to output layer**  

Since the output dimension is $n$, the output layers have $n$ nodes, each taking input from the $M$ output from the hidden layer, the expression for each of the output nodes $f_k$ is given by
$$
f_k = \sigma\left(\sum_{j = 1}^M W^{(2)}_{kj}\text{HiddenNode}_j + b^{(2)}_{k} \right)
$$

Notes: 
 * Yes, this is the same $f_1(\mathbf{x}), \dots, f_n(\mathbf{x})$ from the very begining, but note that the expression above only depends on $\text{HiddenNode}_1, \dots \text{HiddenNode}_M$, which in turn depends on $x_1, \dots, x_d$. 
 * $k$ indexes the output nodes, $k = 1, \dots n$ with $n$, again, being the output dimension. 



**The full unnecessarily messy functional form**
With the definition above, this single-layer-sigmoid neural network is just a function $f = (f_1, \dots, f_n)$, with expression 
$$
f_k(x_1, \dots, x_d) = \sigma\left(\sum_{j = 1}^M 
    W^{(2)}_{kj}\sigma\left(\sum_{i = 1}^d W^{(1)}_{ji}x_i + b^{(1)}_{j}\right) + b^{(2)}_{k} 
\right) \\
$$

<font color="red">
You can see why we choose to draw networks than writing out the functional form.
</font>


## Simplification
Just to simplify the expression for neural network we introduce the following convention  

**Elementwise notation**  
When you use `numpy.log([1, 2, 3])`, numpy internally does `[log(1), log(2), log(3)]` for you, meaning the function is applyed **element-by-element**. The reason is that instead of doing `for elem in array: log(elem)` in python, the loop actually occur in $C$ which is a lot faster. This notation is common and worth knowing: when you have function $\sigma(x)$ that is suppose to take one input and give one output, but the written input is a vector (or array of any dimension) it is understood that we mean apply the function **element-by-element** and output an array of the same dimension. 

**Matrix multiplication**  
Notice that the sum $\sum_{i = 1}^d W_{ji}x_i$ is the same as the dot-product of the $j^{th}$ of the matrix $W$ with the vector $\mathbf{x} = (x_1, \dots, x_d)$. So, in vector form, we have 
$$
\text{Vector output $y_j$ for $j = 1 \dots k$} = \left(\begin{matrix}
y_1 \\
\vdots \\
y_k
\end{matrix}\right) = W\mathbf{x}
$$


**Absorbing the bias term in to the weights**  
The bias term is a little confusing for me. For example is the bias applied before or after the activation? Both should work in practice but I think the consensus is that it is applied before the activation (the expression above is with this assumption). With that in mind, we can do a trick that simplifies the presentation even more. 

We will use the same trick people use in [linear regression](https://en.wikipedia.org/wiki/Linear_regression#Formulation). We add a $0^{th}$ dimension to each layer (including input layer), which is a constant vector of ones. This amounts to adding another node to the input layer so that it becomes $(1, x_1, \dots, x_d)$ instead of just $(x_1, \dots, x_d)$. With this we can rewrite the 

$$
\left(
\begin{matrix}
\sum_{i = 1}^d W_{1i}x_i + b_{1} \\
\dots \\
\sum_{i = 1}^d W_{ki}x_i + b_{k}
\end{matrix}
\right) = 
\left(
\begin{matrix}
W_{11} & W_{12} & \dots & W_{1d} \\
W_{21} & W_{22} & \dots & W_{2d} \\
\ddots & & \\
\dots & W_{ji} & \dots \\
W_{k1} & & \dots & W_{kd}
\end{matrix}
\right)
\left(\begin{matrix} x_1 \\ \vdots \\ x_d \end{matrix}\right) + \left(\begin{matrix} b_1 \\ \vdots \\ b_d \end{matrix}\right) = 
\left(
\begin{matrix}
b_1 & W_{11} & W_{12} & \dots & W_{1d} \\
b_2 & W_{21} & W_{22} & \dots & W_{2d} \\
\ddots & & \\
\dots & W_{ji} & \dots \\
b_k & W_{k1} & & \dots & W_{kd}
\end{matrix}
\right)
\left(\begin{matrix}1 \\ x_1 \\ \vdots \\ x_d \end{matrix}\right) = \widetilde{W}\widetilde{x}
$$
where we absorbed the bias vector into the weight matrix $W$ and $\widetilde{x}$ operation simply add initial element of 1 into a vector. We index those initial addition with the index $0$.


**Pulling together**  

So, with the addition of the "bias node" (i.e. a node that just output 1) in the input layer and the hidden-layer, and using elementwise notation for the activation function, we can rewrite 
$$
f(x) = \sigma(W^{(2)}\widetilde{\sigma(W^{(1)}\tilde{x})})
$$
where the bias vectors for both layers $b^{(1)}$ and $b^{(2)}$ has been absorbed into the weight matrix $W^{(1)}$ and $W^{(2)}$ as the respective layers. 



**Shorthand for the full network**  

We shall write our network as $f_W(x)$ where $W$ include both $W^{(1)}$ and $W^{(2)}$ (or more if this is multi-layered) with the bias vectors absorbed. This is to make the explicit that an network prediction on an input $x$ is dependent on the set of **network parameters** $W$.

----
# Loss function
---
Instead of squared-error, we are prompted to use quartic $x^4$ error. 

Let's expand definitions to see what that means: 
 1. Given an input $x$. Remember that we have $d$ dimensional input so it is really $x = (x_1, \dots, x_d)$. 
 2. with the true output $y(x)$, Remember that we have $n$ dimensional output so it is really $y = (y_1, \dots, y_n)$.
 3. We can compute the network prediction by just passing $x$ through our network $f_W$, giving $f_W(x)$. Again, don't forget that our output is $n$-dimensional, so it is really $f_W(x) = (f_W(x)_1, \dots, f_W(x)_n)$. 
 4. We want out error to be proportional to the difference between $y$ and $f(x)$ to the 4th power. We simply sum the difference component-wise and raise it to the 4th power:
 $$
 L_W(x) = \frac{1}{4} \sum_{k = 1}^n (y_k(x) - f_W(x)_k)^4
 $$

Note: 
 * I am using the same indexing variable $k$ for the output dimension which is the same as the size (number of nodes) in the final output layer. 
 * the $1/4$ is added purely to remove the constant later when we differentiate $x^4$. Scaling the loss by a constant does not change where the minimum occurs. 
 * This is the loss for a single input $x$. So if we are doing online learning, this is enough. 
 * If we are doing batch or minibatch gradient descent, we simply take the batch average: 
 $$ L_W(\{X^1, X^2, \dots, X^b, \dots, X^B\}) = \frac{1}{B}\sum_{b = 1}^B L_W(X_b)$$
 where we use $b$ as the batch sample index with $B$ as the batch size. 

----
# Gradient of the Loss Function
----
Since we aim to use SGD to find the minima of the loss function, we need to compute its gradient **with respect to the network parameters**
$$
\nabla_W L_W(x) 
$$
which tells us the direction of greatest *ascent* when we are at a specific parameter $W$ with $-\nabla_W L_W(x)$ being the direction (in parameter space) where we want to step to get the most decrease in loss. This is really just an exercise in using chain-rule and carefull book keeping of various indices. 

Note: 
 * During calculation of the gradient, the input $x$ or the batch of inputs $\{X^1, \dots, X^B\}$ is **FIXED** and 
 * we only care about minimising loss with respect to the **network parameters**, i.e. the thing that is changing during training time. We are searching for the best(-ish) parameters **GIVEN** that we know the output for the batch of inputs. 

---
## Ok, here goes the Gory details...
First, lets note that, since taking derivative commutes with taking (finite) average, 
$$ \partial_{w} L_W(\{X^1, X^2, \dots, X^b, \dots, X^B\}) = \frac{1}{B}\sum_{b = 1}^B \partial_{w} L_W(X_b)$$
where $\partial_w = \frac{\partial}{\partial w}$ is the partial derivative with respect to the variable $w$. So we really only need to compute the gradient for the per-input loss $L_W(x)$. 

Lets use the notation $\partial_{ji} = \partial_{w_{ji}}$ which is the partial derivative with respect to network weight $W^{(1)}_{ji}$ where (recall that) $i = 0, 1 \dots d$ and $j = 0, 1 \dots M$, where we absorbed bias terms into the $0^{th}$ index. Similarly for the second layer $\partial_{kj}$ is for the network weight $W^{(2)}_{kj}$, $k = 1 \dots n$. With this notation the full gradient is just the vector
$$
\nabla_W L_W(x) = (\partial_{ji} L_W, \partial_{kj} L_W. 
$$
Not the best notation I've come up with but yea .... 



So here goes the chain rule 😭 and lets to the weights for the final layer first... 
$$
\begin{align*}
\partial_{kj} L_W(x) 
& = \frac{1}{4} \sum_{k' = 1}^n \partial_{kj}(y_k(x) - f_W(x)_{k'})^4 \\
& = \sum_{k' = 1}^n (y_{k'}(x) - f_W(x)_{k'})^3 \partial_{kj}f_W(x)_{k'} \\
\end{align*}
$$
Notice that I have to rename the summation variable to $k'$ because that' different from $k$  but both are indexing the output dimension. Fortunately, the variable $w_{kj}$ only appears in terms where $k' = k$, and so alal other terms with $k' \neq k$ have zero $\partial_{kj}$-derivative, and we have 
$$
\partial_{kj} L_W(x) = (y_k(x) - f_W(x)_k)^3 \partial_{kj}f_W(x)_{k} \\
$$
So, as expected we need to know the gradient with respect to the network itself $\partial_{kj}f_W(x)$. 


And that's another chain rule 😭... (we use $h_j$ for the output of $j^{th}$ hidden node (note that it is a scalar)
$$
\begin{align*}
& \partial_{kj}f_W(x)_{k} \\
&= \partial_{kj} \sigma\left(\sum_{j' = 0}^M W^{(2)}_{kj'}h_{j'} \right) \tag*{notice index starts from 0}\\
&= \sigma'\left(\sum_{j' = 0}^M W^{(2)}_{kj'}h_{j'}\right)\, 
   \partial_{kj}\sum_{j' = 0}^M W^{(2)}_{kj'}h_{j'} \tag*{chain rule} \\
&= \sigma'\left(\sum_{j' = 0}^M W^{(2)}_{kj'}h_{j'}\right)\, 
   \partial_{kj} W^{(2)}_{kj}h_{j} \tag*{non-$kj$ terms vanishes} \\
&= \sigma'\left(\sum_{j' = 0}^M W^{(2)}_{kj'}H_{j'}\right)\, h_{j}\tag*{$\partial_{kj}W_{kj} =1$} \\
\end{align*}
$$

So far we have not restrict ourselve to a particular activation function. As long as we know the derivative $\sigma'$ we are good. For sigmoid activation, we have (exercise) $\sigma'(x) = \sigma(x)(1 - \sigma(x))$. So collecting all these together we have 
$$
\begin{align*}
\partial_{kj} L_W(x) 
&= (y_k(x) - f_W(x)_k)^3 \partial_{kj}f_W(x)_{k} \\
&= (y_k(x) - f_W(x)_k)^3 \sigma'\left(\sum_{j' = 0}^M 
    W^{(2)}_{kj'}\sigma\left(\sum_{i = 0}^d W^{(1)}_{j'i}x_i \right)
\right)\, \sigma\left(\sum_{i = 0}^d W^{(1)}_{ji}x_i \right)
\end{align*}
$$

and using elementwise notation
$$
\partial^{(2)} L_W(x) = (y(x) - f_W(x))^3 \sigma'\left( 
    W^{(2)} \sigma\left(W^{(1)}x \right)
\right)\, \sigma\left( W^{(1)}x \right)
$$
with $\partial^{(2)}$ signalling that we are taking derivatie with respect to weights in $W^{(2)}$ only. Note that the output of the expression above is a vector of dimensin $n$ (output dimension). 




In [53]:
def sigmoid(x):
    return 1 / (1 - np.exp(-x))

def sigmoid_diff(x):
    return sigmoid(x) * (1 - sigmoid(x))

def ext(x):
    # extend the vector x by prepending a 1.
    return np.concatenate([[1], x])

class SingleLayerSigmoidFeedForwardNetwork():
    
    def __init__(self, hidden_layer_size, input_dim=1, output_dim=1):
        self.hidden_layer_size = hidden_layer_size
        self.input_dim = input_dim
        self.output_dim = output_dim
        
        self.w1 = np.random.rand(input_dim + 1, hidden_layer_size)
        self.w2 = np.random.rand(hidden_layer_size + 1, output_dim)
        
        self.activation = sigmoid
        self.learning_rate = 0.0001
    
    def forward_pass(self, x):
        h = sigmoid(ext(x) @ self.w1) # hidden layer outputs
        network_output = sigmoid(ext(h) @ self.w2) 
        return network_output
    
    def loss(self, x, y):
        pred = self.forward_pass(x)
        return np.sum((y - pred)**2)/2
    
    def grad_layer2(self, x):
        raise NotImplementedError
    
    def grad_layer1(self, x):
        raise NotImplementedError
    
    def step(self, x):
        self.w2 -= self.learning_rate * self.grad_layer2(x)
        self.w1 -= self.learning_rate * self.grad_layer1(x)
        return
    
    

In [54]:
x = np.array([3, 2, 5])
clf = SingleLayerSigmoidFeedForwardNetwork(10, input_dim=x.shape[0])

clf.forward_pass(x)

NotImplementedError: 